In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Function to create time-series sequences
def create_sequences(data, seq_length):
    sequences, labels = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

# Load data from MongoDB (Assuming you already have price data)
def load_data_from_mongo():
    from pymongo import MongoClient
    client = MongoClient("mongodb://localhost:27017/")  # Update with your MongoDB URI
    db = client["crypto_db"]
    collection = db["price_data"]
    df = pd.DataFrame(list(collection.find()))
    
    # Assuming 'price' is the target column
    df = df.sort_values(by="timestamp")  # Ensure chronological order
    return df[['price']].values  # Extract price column as NumPy array

# Load data
raw_data = load_data_from_mongo()

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(raw_data)

# Create sequences
SEQ_LENGTH = 30  # Use past 30 timestamps to predict the next price
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape input for LSTM (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], SEQ_LENGTH, 1))
X_test = X_test.reshape((X_test.shape[0], SEQ_LENGTH, 1))

# Define LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation="relu"),
    Dense(1)  # Output layer (price prediction)
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)  # Convert back to original scale

# Evaluate model
mse = tf.keras.losses.MeanSquaredError()
print("MSE:", mse(y_test, predictions).numpy())

# Save model
model.save("crypto_lstm_model.h5")


In [ ]:
import numpy as np
print(np.__version__)  # Should output 1.23.5 or lower
